In [2]:
import pandas as pd
import numpy as np
import pickle

# Load F1 dataset

In [6]:
f1_chunks_df = pd.read_pickle("/Users/oscarluo/Documents/GSB570 Generative AI/f1_chunks.pkl")

f1_chunks_df

,Season,Race Name,Driver,CombinedSummary,ChunkText
0,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,In the 2018 Formula 1 season at Abu Dhabi Gran...
1,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,They completed 54 laps and finished in positio...
2,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,Stint information was unavailable for this race.
3,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,The driver made 1 pit stop(s) total during the...
4,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,Fast lap attempts during the race: 26.
...,...,...,...,...,...
64642,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,Fast lap attempts during the race: 42.
64643,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,The driver changed position 0.6 times.
64644,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,"Aggression score for this stint was 6.45, and ..."
64645,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,The lap time variation for this driver was 0.0...


In [7]:
import boto3
import pandas as pd
import numpy as np
import json
import time
import os
from dotenv import load_dotenv

# Langchain + Bedrock / Cohere / HuggingFace
from langchain.embeddings import BedrockEmbeddings
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain_cohere import CohereEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Prompt + parser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Optional if you're doing text chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

In [8]:
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))

In [9]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

In [10]:
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)

In [23]:
query = "Which driver was the most aggressive without sacrificing consistency?"

query_vector = generate_titan_vector_embedding(query)

In [ ]:
# Generate Titan embeddings for each chunk
#f1_chunks_df['embedding'] = f1_chunks_df['ChunkText'].apply(generate_vector_embedding)

# Convert embeddings to numpy arrays for cosine similarity
#f1_chunks_df['embedding'] = f1_chunks_df['embedding'].apply(np.array)

KeyboardInterrupt: 

In [13]:
f1_chunks_spacy = pd.read_pickle("/Users/oscarluo/Documents/GSB570 Generative AI/f1_chunks_spacy.pkl")

f1_chunks_spacy.rename(columns={'embedding': 'spacy_embedding'}, inplace=True)

f1_chunks_spacy

,Season,Race Name,Driver,CombinedSummary,ChunkText,spacy_embedding
0,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,In the 2018 Formula 1 season at Abu Dhabi Gran...,"[-0.71084875, 0.30140305, 0.15889479, -0.01412..."
1,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,They completed 54 laps and finished in positio...,"[-0.7356701, 0.39230585, -0.035814635, 0.05517..."
2,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,Stint information was unavailable for this race.,"[-0.764055, 0.21767625, -0.045777623, 0.051274..."
3,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,The driver made 1 pit stop(s) total during the...,"[-0.64970136, 0.32288375, -0.031953678, -0.054..."
4,2018,Abu Dhabi Grand Prix,Brendon Hartley,In the 2018 Formula 1 season at Abu Dhabi Gran...,Fast lap attempts during the race: 26.,"[-0.7147189, 0.08673333, -0.096873224, 0.05950..."
...,...,...,...,...,...,...
64642,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,Fast lap attempts during the race: 42.,"[-0.7194411, 0.07406334, -0.11782101, 0.085152..."
64643,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,The driver changed position 0.6 times.,"[-0.70173997, 0.22776243, -0.13747771, -0.0362..."
64644,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,"Aggression score for this stint was 6.45, and ...","[-0.7651694, 0.4485487, -0.07458293, 0.0257949..."
64645,2024,United States Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at United States ...,The lap time variation for this driver was 0.0...,"[-0.654046, 0.17290509, -0.1426601, -0.0475678..."


In [14]:
import spacy
nlp = spacy.load("en_core_web_md")

query_vec_spacy = nlp(query).vector
f1_chunks_spacy['spacy_score'] = f1_chunks_spacy['spacy_embedding'].apply(lambda x: cosine_similarity(query_vec_spacy, x))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/Users/oscarluo/Documents/GSB570 Generative AI/gsb570env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
 

In [15]:
# Filtering by top 1000 using spaCy embedding from chunking assignment
top_chunks = f1_chunks_spacy.sort_values(by='spacy_score', ascending=False).head(1000)

In [16]:
# Embedding
top_chunks['embedding'] = top_chunks['ChunkText'].apply(generate_vector_embedding)
top_chunks['embedding'] = top_chunks['embedding'].apply(np.array)

In [31]:
top_chunks.head()

,Season,Race Name,Driver,CombinedSummary,ChunkText,spacy_embedding,spacy_score,embedding,score
27748,2021,Hungarian Grand Prix,Lance Stroll,In the 2021 Formula 1 season at Hungarian Gran...,The driver made 0 pit stop(s) total during the...,"[-0.64948505, 0.3011401, -0.05512409, -0.05155...",0.899902,"[-0.333984375, 0.90234375, 0.2099609375, 0.118...",0.447006
25226,2021,Belgian Grand Prix,Lewis Hamilton,In the 2021 Formula 1 season at Belgian Grand ...,The driver made 0 pit stop(s) total during the...,"[-0.64948505, 0.3011401, -0.05512409, -0.05155...",0.899902,"[-0.333984375, 0.90234375, 0.2099609375, 0.118...",0.447006
60065,2024,Japanese Grand Prix,Alexander Albon,In the 2024 Formula 1 season at Japanese Grand...,The driver made 0 pit stop(s) total during the...,"[-0.64948505, 0.3011401, -0.05512409, -0.05155...",0.899902,"[-0.333984375, 0.90234375, 0.2099609375, 0.118...",0.447006
60055,2024,Italian Grand Prix,Yuki Tsunoda,In the 2024 Formula 1 season at Italian Grand ...,The driver made 0 pit stop(s) total during the...,"[-0.64948505, 0.3011401, -0.05512409, -0.05155...",0.899902,"[-0.333984375, 0.90234375, 0.2099609375, 0.118...",0.447006
6704,2018,United States Grand Prix,Fernando Alonso,In the 2018 Formula 1 season at United States ...,The driver made 0 pit stop(s) total during the...,"[-0.64948505, 0.3011401, -0.05512409, -0.05155...",0.899902,"[-0.333984375, 0.90234375, 0.2099609375, 0.118...",0.447006


In [33]:
top_chunks.to_csv("/Users/oscarluo/Documents/GSB570 Generative AI/top_chunks.csv", index=False)

In [24]:
top_chunks['score'] = top_chunks['embedding'].apply(lambda x: cosine_similarity(query_vector, x))

# Select top N
retrieved = top_chunks.sort_values(by='score', ascending=False).head(5)

context = "\n\n".join(retrieved['ChunkText'])

In [29]:
from langchain_community.chat_models import BedrockChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = BedrockChat(
    model_id="meta.llama3-8b-instruct-v1:0",
    region_name="us-west-2"
)

prompt = f"""
You are an expert Formula 1 analyst.

Question: {query}

Use the following race data to support your answer:

{context}

Answer:
"""

In [30]:
chat_prompt = ChatPromptTemplate.from_template("{question}")
chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"question": prompt})
print(response)

 
The driver who was the most aggressive without sacrificing consistency is Lewis Hamilton. He made 4 pit stops during the race, which is the same number as the other drivers, but his average pit stop time was 310.0 seconds, which is the lowest among all the drivers. This suggests that he was able to make aggressive moves on track without sacrificing consistency, as he was able to maintain a consistent pace throughout the race.

It's worth noting that the other drivers may have also been aggressive, but their pit stop data suggests that they may have sacrificed some consistency for the sake of aggression. For example, Max Verstappen made 3 pit stops, which is one fewer than Hamilton, but his average pit stop time was 505.4 seconds, which is significantly longer than Hamilton's. This suggests that Verstappen may have been more aggressive on track, but at the cost of consistency.

Overall, Hamilton's pit stop data suggests that he was able to strike a balance between aggression and consi

This trained on a sample using the top 1000 chunks based on spaCy's vector cosine similarity that I had in the chunking assignment. I tried to embed using Titan but that would have taken too long so I decided to just do a small sample baassed on the top chunks from spaCy embedding.

Average pit stop time also does not seem correct. The average pit stop time in F1 is ~2.5 seconds. Going back to check the F1 dataset from Kaggle, it seems like the dataset is incorrect with average pit time being 120.109095, min is 17.358500 and max is 3055.732000. There was no unit specified on Kaggle, so these values suggest either a unit mismatch (e.g., milliseconds or another scale) or a mislabeled/aggregated field.